# Notebook 1: Querying Pushshift API

Welcome to my project. In this project I am searching for trends between the Call of Duty: Black Ops 4 subreddit activity and the release of patches on the game itself. I intend to find trends that might suggest to the developers that they should take user feedback into account when releasing patches. I will do this by trying to predict when patches will be released based on word usage, sentiment, and user churn. If I can find these trends, it would suggest that perhaps the best course of action is to look at what is talked about online and have those topics influence the developers. While I understand that the company is most interested in making profit while following their own 'story', or direction for the game, I think that examining the front page of the internet for feedback and suggestions might prove to be worthwhile. The online reddit participation, as well as the sentiment of comments each day, might be a useful thing to examine when preparing new patches. Over time, participation will decrease naturally, but if the company can continue to provide a game that encourages positive sentiment regardless of user dropoff, the game can still be a huge success. It is the combination of user dropoff and sentiment dropoff that will lead to the game dying entirely. 

In this notebook, I will primarily be querying the pushshift api for comments, score, post, date, and other metadata. I will go back until the day before the game was released, and examine commentary from then until present day. 

In [1]:
import pandas as pd
import requests
import json
import time
import re
import matplotlib.pyplot as plt

%matplotlib inline

API url.

In [7]:
url = 'https://api.pushshift.io/reddit/search/'

Initial parameters I am examining just to test out the query. 

In [8]:
params = {'searchType' : 'comment', 
          'subreddit' : 'blackops4',
          'sort' : 'desc',
          'size' : 100,
          'before': '193d',
          'after' : '194d'
         }

In [9]:
response = requests.get(url, params = params)

In [10]:
response.status_code

200

In [11]:
len(response.json()['data'])

100

Examining the created_utc time of my initial post. I want to go back to the day before the game came out, which would be October 11, 2018.

In [12]:
response.json()['data'][1]['created_utc']

1540079647

In [13]:
[time.asctime(time.gmtime(response.json()['data'][i]['created_utc'])) for i in range(len(response.json()['data']))]

['Sat Oct 20 23:54:14 2018',
 'Sat Oct 20 23:54:07 2018',
 'Sat Oct 20 23:54:06 2018',
 'Sat Oct 20 23:54:05 2018',
 'Sat Oct 20 23:54:04 2018',
 'Sat Oct 20 23:53:58 2018',
 'Sat Oct 20 23:53:57 2018',
 'Sat Oct 20 23:53:54 2018',
 'Sat Oct 20 23:53:53 2018',
 'Sat Oct 20 23:53:52 2018',
 'Sat Oct 20 23:53:49 2018',
 'Sat Oct 20 23:53:49 2018',
 'Sat Oct 20 23:53:46 2018',
 'Sat Oct 20 23:53:45 2018',
 'Sat Oct 20 23:53:44 2018',
 'Sat Oct 20 23:53:37 2018',
 'Sat Oct 20 23:53:29 2018',
 'Sat Oct 20 23:53:27 2018',
 'Sat Oct 20 23:53:27 2018',
 'Sat Oct 20 23:53:24 2018',
 'Sat Oct 20 23:53:22 2018',
 'Sat Oct 20 23:53:21 2018',
 'Sat Oct 20 23:52:45 2018',
 'Sat Oct 20 23:52:38 2018',
 'Sat Oct 20 23:52:37 2018',
 'Sat Oct 20 23:52:33 2018',
 'Sat Oct 20 23:52:31 2018',
 'Sat Oct 20 23:52:26 2018',
 'Sat Oct 20 23:52:18 2018',
 'Sat Oct 20 23:52:17 2018',
 'Sat Oct 20 23:52:16 2018',
 'Sat Oct 20 23:52:14 2018',
 'Sat Oct 20 23:52:13 2018',
 'Sat Oct 20 23:52:00 2018',
 'Sat Oct 20 2

This function will create a logfile and format the file names with a unique timestamp.

In [14]:
def filename_format_log(file_path, 
                        logfile = '../data/file_log.txt',  
                        file_description = None,
                        now = round(time.time())): 
   
    try:
        ext = re.search('(?<!^)(?<!\.)\.(?!\.)', file_path).start() 
    except:
        raise NameError('Please enter a relative path with a file extension.') 
    
    stamp = re.search('(?<!^)(?<!\.)[a-z]+_[a-z]+(?=\.)', file_path).start()
    formatted_name = f'{file_path[:stamp]}{round(time.time())}_{file_path[stamp:]}'
    #now = now
    if not file_description:
        file_description = f'Pull: {time.asctime(time.gmtime(round(time.time())))}'
    with open(logfile, 'a+') as f:
        f.write(f'{formatted_name}: {file_description}\n')
    return formatted_name, now, file_description

This function will collect comments and parse them into a dataframe with the features of interest, saving out the raw data for each pull. After saving each query, which max out at 1000 posts, it runs a time delay and then continues for as many queries as I set.
This one is specifically set to pull comments, and I can set a 'before' time that resets to the new 'before' time after each posts. This way, I can query in chunks back to October 11. 

In [15]:
def reddit_query(subreddits, n_samples=50000, searchType='comments', before=None, after=None):
    url = f'https://api.pushshift.io/reddit/search/'
    last_comment = round(time.time())
    comment_list = []

    run = 1
    while len(comment_list) < n_samples:

        try:
            print(f'Starting query {run}')

            params = {'searchType':searchType,
              'subreddit':subreddits,
              'sort':'desc',
              'size':1000,
              'before':before,
              'after':after,
             }

            response = requests.get(url, params = params)
            posts = response.json()['data']

            if len(posts) == 0:
                last_comment = last_comment
            else:
                last_comment = posts[-1]['created_utc']
                before = posts[-1]['created_utc']
                comment_list.extend(posts)
                timestamp = posts[-1]['created_utc']
                time.sleep(.5)
                run += 1
        except:
            if response.status_code != 200:
                return f'Check status. Error code: {response.status_code}'
            else:
                return 'Error. Pull not completed.'

    formatted_name, now, file_description = filename_format_log(file_path =f'../assets/raw_{searchType}s.json', now=timestamp)
    with open(formatted_name, 'w+') as f:
        json.dump(comment_list, f)

    print(f'Saved and completed query and returned {len(comment_list)} {searchType}s.')
    print(f'Reddit text is ready for processing.')
    #return print(f'Last timestamp was {timestamp}.')

# WARNING THIS CELL TAKES A LONG TIME. 

Running the function to pull 2 million comments. This should take me back to the start date. 



In [16]:
reddit_query(subreddits = 'blackops4', n_samples=2_000_000, before = None, after = None)

Starting query 1
Starting query 2
Starting query 3
Starting query 4
Starting query 5
Starting query 6
Starting query 7
Starting query 8
Starting query 9
Starting query 10
Starting query 11
Starting query 12
Starting query 13
Starting query 14
Starting query 15
Starting query 16
Starting query 17
Starting query 18
Starting query 19
Starting query 20
Starting query 21
Starting query 22
Starting query 23
Starting query 24
Starting query 25
Starting query 26
Starting query 27
Starting query 28
Starting query 29
Starting query 30
Starting query 31
Starting query 32
Starting query 33
Starting query 34
Starting query 35
Starting query 36
Starting query 37
Starting query 38
Starting query 39
Starting query 40
Starting query 41
Starting query 42
Starting query 43
Starting query 44
Starting query 45
Starting query 46
Starting query 47
Starting query 48
Starting query 49
Starting query 50
Starting query 51
Starting query 52
Starting query 53
Starting query 54
Starting query 55
Starting query 56
S

In this next cell, I am opening up my json file, using the name with the unique timestamp in the assets folder. 

In [18]:
with open(f'../assets/1556758929_raw_commentss.json', 'r') as f:
    blackops_comments = json.load(f)

Making sure my query pulled the correct number of comments.

In [19]:
len(blackops_comments)

2000000

Examining one value in the json. 

In [20]:
blackops_comments[1]

{'all_awardings': [],
 'author': 'Only-One-Gender',
 'author_flair_background_color': 'transparent',
 'author_flair_css_class': None,
 'author_flair_richtext': [{'a': ':PlayStation:',
   'e': 'emoji',
   'u': 'https://emoji.redditmedia.com/s8auodloebd11_t5_2untl/PlayStation'}],
 'author_flair_template_id': 'f415fed0-94e3-11e8-984f-0ef2d2019e32',
 'author_flair_text': ':PlayStation:',
 'author_flair_text_color': 'dark',
 'author_flair_type': 'richtext',
 'author_fullname': 't2_37ubeoa5',
 'author_patreon_flair': False,
 'body': "Yeah I'm getting them",
 'created_utc': 1556754886,
 'gildings': {},
 'id': 'em9spes',
 'is_submitter': True,
 'link_id': 't3_bjnzy5',
 'locked': False,
 'no_follow': True,
 'parent_id': 't1_em9so2y',
 'permalink': '/r/Blackops4/comments/bjnzy5/this_is_so_damn_satisying/em9spes/',
 'retrieved_on': 1556754887,
 'score': 1,
 'send_replies': True,
 'stickied': False,
 'subreddit': 'Blackops4',
 'subreddit_id': 't5_2untl',
 'total_awards_received': 0}

Parsing the data to create a pandas dataframe.

In [5]:
def reddit_parse(sample, df):
    
    col_list = ['author',
                'body',
                'created_utc',
                'score',
                'permalink',
                #'gildings'
                ]
    
    df = pd.DataFrame(sample)
    df = df[col_list]
    
    df.rename(columns={'subreddit':'subreddit'}, inplace=True)
    
    col_order = ['author',
                'body',
                'created_utc',
                'score',
                'permalink',
                #'gildings'
                ]

    return df[col_order]

Creating my comment dataframe. 

In [21]:
df = reddit_parse(blackops_comments, df = 'df')

Checking the shape. 

In [7]:
df.shape

(2000000, 5)

Checking for uniqueness to make sure I didn't query a bunch of duplicates. 

In [8]:
df['body'].nunique()

1887121

1,887,121 unique comments out of 2,000,000 is fine for the purpose of this project. Some of those could be comments that were posted twice by accident or short comments that are trite and not worth reading into. Something like 'lol' or the like. 

Turning my date column to a datetime datatype. 

In [23]:
df['date'] = pd.to_datetime(df['created_utc'], unit='s')

In [25]:
df['date'] = df['date'].dt.date

In [26]:
df.head()

,author,body,created_utc,score,permalink,date
0,Pikachu-Faroo,So they're just gonna ignore the Sparrow bug?,1556754911,1,/r/Blackops4/comments/bjn8sm/may_1st_update_ga...,2019-05-01
1,Only-One-Gender,Yeah I'm getting them,1556754886,1,/r/Blackops4/comments/bjnzy5/this_is_so_damn_s...,2019-05-01
2,twopumpstump,So you are getting the new stuff? I heard the ...,1556754860,1,/r/Blackops4/comments/bjnzy5/this_is_so_damn_s...,2019-05-01
3,MoneyMajesty,Was just about to say this. The sad thing abou...,1556754825,1,/r/Blackops4/comments/bjnygs/if_only_treyarcha...,2019-05-01
4,Slaya420D,*claims the game is dead*\n\nI can find a lobb...,1556754821,1,/r/Blackops4/comments/bjo9k6/seriously_this_ga...,2019-05-01


Pushing my dataframe to csv for interpretation in the next notebook.

In [17]:
df.to_csv('../data/blackops_comments.csv')

# Conclusion and Next Steps:

Now that I queried and fit my data to a dataframe, I can begin the data cleaning and exploratory data analysis process. I foresee some issues with the size of the data, but given that I am specifically looking at timeseries data, once I condense my data down to each date, it should be very manageable to work with. 